In [ ]:
# library imports:

import pandas as pd
from pathlib import Path
import portiloop_software
import torch
from portiloop_software import run_offline_unlabelled, get_final_model_config_dict
from matplotlib import pyplot as plt

In [ ]:
# path to the portiloop software package:

path_software = Path(portiloop_software.__file__).parent.absolute()

In [ ]:
# path to the folder containing pre-trained models:

path_experiments = path_software / 'experiments'

In [ ]:
# path to the non-annotated data, change this to your own for custom applications:

path_data = path_software / 'dataset' / 'example_data_not_annotated.txt'

In [ ]:
# load non-annotated data into a pytorch Tensor:

data = torch.Tensor(pd.read_csv(path_data, header=None).values).squeeze()

In [ ]:
print(f"The data has been loaded as a torch Tensor of {len(data)} data points:\n{data}")

In [ ]:
# configuration dictionary of the model:

config_dict = get_final_model_config_dict()

In [ ]:
# run offline inference (on all data points):

output, true_idx = run_offline_unlabelled(config_dict=config_dict,
                                          path_experiments=path_experiments,
                                          unlabelled_segment=data)

In [ ]:
# convert result for visualization:

out = output.numpy() - 5
idx = true_idx.numpy()
sig = data[idx].numpy()
idx_s = idx / config_dict['fe']
last_tick = idx_s[-1]

In [ ]:
# plot results

start = 115
duration = 10
THRESHOLD = 0.5

stop = start + duration

def y1axtoy2ax(y):
    res = y - 5
    return res

def y1axtoy2ax(y):
    res = y + 5
    return res

plt.rcParams['figure.figsize'] = [15, 4]
fig, ax = plt.subplots()
ax.plot(idx_s, sig, label="Signal")
ax.plot(idx_s, out, label="ANN output")
ax.hlines(-5 + THRESHOLD, 0, last_tick, colors='lightgrey', label="Threshold", linestyles=':')
ax.set_ylim((-5,5))
ax.set_xlim((start, stop))
ax.set_xlabel("Time (s)")
ax.set_ylabel("Signal (arb. unit)")
secy = ax.secondary_yaxis('right', functions=(y1axtoy2ax, y1axtoy2ax))
secy.set_yticks([0,1])
ax.axes.yaxis.set_visible(False)
ax.set_title(f"Threshold {THRESHOLD}")
plt.tight_layout()
plt.legend()
plt.show()